# Fine-Tuning LLMs

In this exercise, you will fine-tune the [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model for enhanced dialogue summarization. You will first explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter-Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

## 1. Set up Dependencies and Load Dataset and LLM

In [1]:
!pip install datasets evaluate rouge_score peft -q

In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset

In [3]:
dataset = load_dataset('knkarthick/dialogsum')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Load the pre-trained [Flan-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of Flan-T5. Setting `torch_dtype=torch.bfloat16` specifies the data type to be used by this model, which can reduce GPU memory usage since `bfloat16` uses half as much memory per number compared to `float32`, the default precision for most models.

In [4]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 2. Test the Model with Zero-Shot Inferencing

Test the model with zero-shot inference.

In [5]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

You can see that the model struggles to summarize the dialogue compared to the baseline summary, and simply repeats the first sentence from the dialogue.

## 3. Perform Full Fine-Tuning

### 3.1 Preprocess the Dataset

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Summarize the following conversation.`, and to the start of the summary with `Summary:` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.
Alice: This is her part of the conversation.
Bob: This is his part of the conversation.    
Summary:
```

Training response (summary):
```
Both Alice and Bob participated in the conversation.
```

**Exercise**: Write a function to tokenize a batch of examples from the dialogue dataset. The function should concatentate the dialogues with the predefined prompt, tokenize them along with their summaries, and define the tokenized summaries as the labels.

In [12]:
def tokenize(examples):
    ### WRITE YOUR CODE HERE
      inputs = [f"Summarize the following conversation.\n{d}\nSummary:\n" for d in examples["dialogue"]]
      model_inputs = tokenizer(
          inputs,
          max_length=512,
          truncation=True,
          padding = 'max_length'
      )
      with tokenizer.as_target_tokenizer():
        targets = tokenizer(
            examples["summary"],
            max_length=128,
            truncation=True,
            padding = 'max_length'
        )
      model_inputs["labels"] = targets["input_ids"]
      return model_inputs

In [13]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

### 3.2 Fine-Tune the Model

**Exercise**: Utilize the Hugging Face Trainer API for training the model on the preprocessed dataset. Define the training arguments, a data collator, and create a `Seq2SeqTrainer` instance. Train the model for one epoch.

In [16]:
### WRITE YOUR CODE HERE
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-base-dialogsum-checkpoint",
    overwrite_output_dir=True,
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=50,
    predict_with_generate=True,
    report_to="none",
    bf16=torch.cuda.is_available(),
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=original_model)

trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)



/tmp/ipython-input-33067838.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Training a fully fine-tuned version of the model should take about 10 minutes on a Google Colab GPU machine.

In [17]:
trainer.train()

Step,Training Loss
50,38.855100
100,38.687700
150,36.906300
200,33.329000
250,26.534100
300,12.444900
350,4.049600


TrainOutput(global_step=390, training_loss=24.822152514335436, metrics={'train_runtime': 459.9277, 'train_samples_per_second': 27.091, 'train_steps_per_second': 0.848, 'total_flos': 8532076611502080.0, 'train_loss': 24.822152514335436, 'epoch': 1.0})

Save the model to a local folder:

In [18]:
model_path = './flan-t5-base-dialogsum-checkpoint'

original_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json',
 './flan-t5-base-dialogsum-checkpoint/special_tokens_map.json',
 './flan-t5-base-dialogsum-checkpoint/spiece.model',
 './flan-t5-base-dialogsum-checkpoint/added_tokens.json',
 './flan-t5-base-dialogsum-checkpoint/tokenizer.json')

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

In [19]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-checkpoint',
                                                       torch_dtype=torch.bfloat16)

Reload the original Flan-T5-base model:

In [20]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 3.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Section 2, using the original model and the fully fine-tuned model.

In [22]:
### WRITE YOUR CODE HERE
index = 56
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')

orig_ids = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(orig_ids, skip_special_tokens=True)

instr_ids = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(instr_ids, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'ORIGINAL MODEL (zero-shot) SUMMARY:\n{original_model_summary}\n')
print(dash_line)
print(f'INSTRUCT MODEL (full FT) SUMMARY:\n{instruct_model_summary}\n')


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Is anybody in?
#Person2#: How can I help you?
#Person1#: I have a headache.
#Person2#: Let me take your temperature with a thermometer.
#Person1#: OK.
#Person2#: I think you have a small fever.
#Person1#: I thought so. I felt dizzy this morning.
#Person2#: You should've called in sick! Next time, have either of your parents call the school office.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# checks #Person1#'s physical condition and finds #Person1# has a fever.
----------------------------------------------------------------------------------------------------
ORIGINAL MODEL (zero-shot) SUMMARY:
#Person1#: I have a headache. #Person2#: I think you have a small fever.

----------------------------------------------------------------------------------------------------
INSTRU

The fine-tuned model is able to create a much better summary of the dialogue compared to the original model.

### 3.4 Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [23]:
rouge = evaluate.load('rouge')

**Exercise**: Generate the outputs for a sample of the test set with the fine-tuned model (use only the first 10 dialogues and summaries to save time).

In [25]:
### WRITE YOUR CODE HERE
num_examples = 10
test_dialogues = dataset['test']['dialogue'][:num_examples]
human_baseline_summaries = dataset['test']['summary'][:num_examples]

original_model_summaries = []
instruct_model_summaries = []

for dlg in test_dialogues:
    text_input = f"Summarize the following conversation.\n{dlg}\nSummary:\n"
    tokens = tokenizer(text_input, return_tensors='pt')

    orig_result = original_model.generate(tokens['input_ids'], max_new_tokens=50)[0]
    original_model_summaries.append(tokenizer.decode(orig_result, skip_special_tokens=True))

    inst_result = instruct_model.generate(tokens['input_ids'], max_new_tokens=50)[0]
    instruct_model_summaries.append(tokenizer.decode(inst_result, skip_special_tokens=True))


Evaluate the models computing ROUGE metrics:

In [26]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)]
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)]
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.22420174087325248), 'rouge2': np.float64(0.09872463768115944), 'rougeL': np.float64(0.2089424816139932), 'rougeLsum': np.float64(0.21333914728682168)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.3517586339059473), 'rouge2': np.float64(0.13560846560846562), 'rougeL': np.float64(0.30331246693203817), 'rougeLsum': np.float64(0.3050757898077988)}


The results show substantial improvement in all ROUGE metrics:

In [27]:
print("Absolute percentage improvement of the instruct model over the original model:")

for key in instruct_model_results:
    improvement = instruct_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the instruct model over the original model:
rouge1: 12.76%
rouge2: 3.69%
rougeL: 9.44%
rougeLsum: 9.17%


## 4. Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** instead of "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning, with comparable evaluation results as you will see soon.

One of the most popular PEFT methods is **Low-Rank Adaptation (LoRA)**, which  introduces low-rank matrices to adapt the LLM with minimal additional parameters. In most cases, when someone says PEFT, they typically mean LoRA.  After fine-tuning for a specific task with LoRA, the result is that the original LLM remains unchanged and a newly-trained "LoRA adapter" emerges. This LoRA adapter is much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

At inference time, the LoRA adapter is reunited and combined with its original LLM to serve the inference request. The benefit is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 4.1 Setup the LoRA model for Fine-Tuning

You first need to define the configuration of the LoRA model. Have a look at the configuration below. The key configuration element to adjust is the rank (`r`) of the adapter, which influences its capacity and complexity. Experiment with various ranks, such as 8, 16, or 32, and see how they affect the results.

In [28]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)

Add LoRA adapter layers/parameters to the original LLM to be trained:

In [29]:
peft_model = get_peft_model(original_model, lora_config)

The number of trainable model parameters in the LoRA model is:

In [30]:
peft_model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093


### 4.2 Train the LoRA Adapter

**Exercise**: Define training arguments and create a `Seq2SeqTrainer` instance for the LoRA model. Use a higher learning rate than full fine-tuning (e.g., `1e-3`).

In [31]:
### WRITE YOUR CODE HERE
peft_training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-base-dialogsum-lora",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy='no',
    predict_with_generate=True,
    learning_rate=1e-3,
    fp16=False,
    report_to='none'
)

peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)


/tmp/ipython-input-568516899.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  peft_trainer = Seq2SeqTrainer(


Train the PEFT adapter. Training should take about 6 minutes on a Google Colab GPU machine.

In [32]:
peft_trainer.train()

Step,Training Loss
10,39.797900
20,40.254500
30,37.973600
40,36.292000
50,32.384100
60,27.365300
70,22.526200
80,17.410400
90,10.131200
100,4.865800


TrainOutput(global_step=390, training_loss=7.7250482277992445, metrics={'train_runtime': 578.4609, 'train_samples_per_second': 21.54, 'train_steps_per_second': 0.674, 'total_flos': 8667537195663360.0, 'train_loss': 7.7250482277992445, 'epoch': 1.0})

Save the model to a local folder:

In [33]:
peft_model.save_pretrained('./flan-t5-base-dialogsum-lora')

Load the PEFT model:

In [34]:
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer

peft_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

Reload the original Flan-T5-base model:

In [35]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 4.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Sections 2 and 3, using the original model, the fully fine-tuned model and the PEFT model.

In [36]:
### WRITE YOUR CODE HERE
test_idx = 56
line_break = '-' * 100

conversation = dataset['test'][test_idx]['dialogue']
reference_summary = dataset['test'][test_idx]['summary']

prompt_text = f"Summarize the following conversation.\n{conversation}\nSummary:\n"
encoded_input = tokenizer(prompt_text, return_tensors='pt')

baseline_out = original_model.generate(encoded_input['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(baseline_out, skip_special_tokens=True)

finetuned_out = instruct_model.generate(encoded_input['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(finetuned_out, skip_special_tokens=True)

peft_out = peft_model.generate(encoded_input['input_ids'], max_new_tokens=50)[0]
peft_model_summary = tokenizer.decode(peft_out, skip_special_tokens=True)

print(line_break)
print(f'INPUT PROMPT:\n{conversation}')
print(line_break)
print(f'BASELINE HUMAN SUMMARY:\n{reference_summary}')
print(line_break)
print(f'ORIGINAL MODEL SUMMARY:\n{original_model_summary}')
print(line_break)
print(f'INSTRUCT MODEL SUMMARY:\n{instruct_model_summary}')
print(line_break)
print(f'PEFT MODEL SUMMARY:\n{peft_model_summary}\n')


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Is anybody in?
#Person2#: How can I help you?
#Person1#: I have a headache.
#Person2#: Let me take your temperature with a thermometer.
#Person1#: OK.
#Person2#: I think you have a small fever.
#Person1#: I thought so. I felt dizzy this morning.
#Person2#: You should've called in sick! Next time, have either of your parents call the school office.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# checks #Person1#'s physical condition and finds #Person1# has a fever.
----------------------------------------------------------------------------------------------------
ORIGINAL MODEL SUMMARY:
#Person1#: I have a headache. #Person2#: I think you have a small fever.
----------------------------------------------------------------------------------------------------
INSTRUCT MODEL SUMM

### 4.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Exercise**: Generate the outputs for a sample of the test set with the PEFT model (use only the first 10 dialogues and summaries to save time).

In [37]:
### WRITE YOUR CODE HERE
n_samples = 10
eval_dialogues = dataset['test']['dialogue'][:n_samples]
human_baseline_summaries = dataset['test']['summary'][:n_samples]

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for conversation_text in eval_dialogues:
    formatted_prompt = f"Summarize the following conversation.\n{conversation_text}\nSummary:\n"
    input_ids = tokenizer(formatted_prompt, return_tensors='pt')

    baseline_gen = original_model.generate(input_ids['input_ids'], max_new_tokens=50)[0]
    original_model_summaries.append(tokenizer.decode(baseline_gen, skip_special_tokens=True))

    finetuned_gen = instruct_model.generate(input_ids['input_ids'], max_new_tokens=50)[0]
    instruct_model_summaries.append(tokenizer.decode(finetuned_gen, skip_special_tokens=True))

    peft_gen = peft_model.generate(input_ids['input_ids'], max_new_tokens=50)[0]
    peft_model_summaries.append(tokenizer.decode(peft_gen, skip_special_tokens=True))



Compute ROUGE score for this subset of the data.

In [38]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.22420174087325248), 'rouge2': np.float64(0.09872463768115944), 'rougeL': np.float64(0.2089424816139932), 'rougeLsum': np.float64(0.21333914728682168)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.3517586339059473), 'rouge2': np.float64(0.13560846560846562), 'rougeL': np.float64(0.30331246693203817), 'rougeLsum': np.float64(0.3050757898077988)}
PEFT MODEL:
{'rouge1': np.float64(0.3798405725048174), 'rouge2': np.float64(0.12038713332191245), 'rougeL': np.float64(0.29900923638504723), 'rougeLsum': np.float64(0.30010104805388926)}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Calculate the improvement of PEFT over the original model:

In [39]:
print("Absolute percentage improvement of the PEFT model over the original model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the original model:
rouge1: 15.56%
rouge2: 2.17%
rougeL: 9.01%
rougeLsum: 8.68%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [40]:
print("Absolute percentage improvement of the PEFT model over the instruct model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - instruct_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the instruct model:
rouge1: 2.81%
rouge2: -1.52%
rougeL: -0.43%
rougeLsum: -0.50%


You can see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources.